In [99]:
!pip install nltk
!python -m nltk.downloader stopwords

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [101]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [102]:
df = pd.read_csv("/content/Tweets.csv")
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [103]:
df = df.drop(columns=['textID','selected_text'])
df.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [104]:
df.shape

(27481, 2)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [106]:
df.isnull().sum()

text         1
sentiment    0
dtype: int64

In [107]:
df.dropna(inplace=True)

In [108]:
df.duplicated().sum()

0

In [109]:
df.describe()

,text,sentiment
count,27480,27480
unique,27480,3
top,"I`d have responded, if I were going",neutral
freq,1,11117


In [110]:
df['sentiment'].value_counts()

sentiment
neutral     11117
positive     8582
negative     7781
Name: count, dtype: int64

In [111]:
def drop_row_func(category,num):
  global df

  random_indices = np.random.choice(category.index, num, replace=False)

  df = df.drop(random_indices)

cat1 = df[df['sentiment'] == "neutral"]
cat2 = df[df['sentiment'] == "positive"]

drop_row_func(cat1,3336)
drop_row_func(cat2,801)

In [112]:
df['sentiment'].value_counts()

sentiment
neutral     7781
negative    7781
positive    7781
Name: count, dtype: int64

In [113]:
df['sentiment'] = df['sentiment'].replace({"negative": 0,"neutral": 1,"positive": 2})


In [114]:
df['text'] = df['text'].str.lower()

df['text'] = df['text'].astype(str)

In [115]:
#remove stopwords
import string
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])
df['text'] = df['text'].apply(lambda x: " ".join(term for term in x.split() if term not in stop_words))

In [116]:
df.head()

,text,sentiment
0,"i`d responded, going",1
1,sooo sad miss san diego!!!,0
2,boss bullying me...,0
3,interview! leave alone,0
4,"sons ****, couldn`t put releases already bought",0


In [117]:
x = df['text']
y = df['sentiment']

In [119]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(x)

In [123]:
X_train,x_test,Y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

svm_model = SVC()
svm_model.fit(X_train,Y_train)

SVC()

In [124]:
y_pred = svm_model.predict(x_test)

print("Classification Repost :")
print(classification_report(y_test,y_pred))
print("Accuracy Score --> ",accuracy_score(y_test,y_pred))

Classification Repost :
              precision    recall  f1-score   support

           0       0.74      0.62      0.67      1576
           1       0.54      0.72      0.62      1545
           2       0.81      0.69      0.74      1548

    accuracy                           0.67      4669
   macro avg       0.70      0.67      0.68      4669
weighted avg       0.70      0.67      0.68      4669

Accuracy Score -->  0.6729492396658814


In [125]:
lg_model = LogisticRegression()
lg_model.fit(X_train,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [126]:
y_pred = lg_model.predict(x_test)

print("Classification Repost :")
print(classification_report(y_test,y_pred))
print("Accuracy Score --> ",accuracy_score(y_test,y_pred))

Classification Repost :
              precision    recall  f1-score   support

           0       0.71      0.66      0.69      1576
           1       0.57      0.66      0.61      1545
           2       0.80      0.72      0.76      1548

    accuracy                           0.68      4669
   macro avg       0.69      0.68      0.68      4669
weighted avg       0.69      0.68      0.68      4669

Accuracy Score -->  0.6815163846648105


In [127]:
from sklearn.naive_bayes import MultinomialNB

naive = MultinomialNB()
naive.fit(X_train,Y_train)

MultinomialNB()

In [128]:
y_pred = naive.predict(x_test)

print("Classification Repost :")
print(classification_report(y_test,y_pred))
print("Accuracy Score --> ",accuracy_score(y_test,y_pred))

Classification Repost :
              precision    recall  f1-score   support

           0       0.66      0.72      0.69      1576
           1       0.57      0.48      0.52      1545
           2       0.70      0.75      0.72      1548

    accuracy                           0.65      4669
   macro avg       0.64      0.65      0.64      4669
weighted avg       0.64      0.65      0.64      4669

Accuracy Score -->  0.6485328764189334


In [129]:
svm2_model = SVC(C = 1.0,kernel='linear',degree=3,gamma='auto')
svm2_model.fit(X_train,Y_train)

y_pred = svm2_model.predict(x_test)

print("Classification Repost :")
print(classification_report(y_test,y_pred))
print("Accuracy Score --> ",accuracy_score(y_test,y_pred))

Classification Repost :
              precision    recall  f1-score   support

           0       0.72      0.66      0.69      1576
           1       0.57      0.68      0.62      1545
           2       0.81      0.72      0.76      1548

    accuracy                           0.69      4669
   macro avg       0.70      0.69      0.69      4669
weighted avg       0.70      0.69      0.69      4669

Accuracy Score -->  0.6862283144142215
